## Import dependancies and set environment determinism

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''  # Hide GPU devices

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
#     tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(
    0
)
import numpy as np
import random
import pandas as pd


SEED = 378
# SEED = 123
print(SEED)
def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

def set_global_determinism(seed=SEED):
    set_seeds(seed=seed)

    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

# Call the above function with seed value
set_global_determinism(seed=SEED)

In [ ]:
import glacierml as gl
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tqdm import tqdm
import path_manager as pm
[
        home_path, data_path, RGI_path, glathida_path, 
        ref_path, coregistration_testing_path, 
        arch_test_path, LOO_path
] = pm.set_paths()

In [ ]:
def run_model(model_path, n,l1,l2,loss):
            
    normalizer = preprocessing.Normalization(axis=-1)
    normalizer.adapt(np.array(trfeat[n]))

    model = gl.build_dnn_model(
        normalizer, learning_rate = 0.01, 
        layer_1 = l1, layer_2 = l2,loss = loss
    )
    model_history = model.fit(
        trfeat[n],
        trlabs[n],
        validation_split=0.2,
        callbacks = [callback],
        verbose=0, 
        epochs=500
    )
    model_filename = os.path.join(model_path)
    model.save(model_filename)
    return model

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    min_delta = 0.001,
    patience = 10,
    verbose = 0,
    mode = 'auto',
    baseline = None,
    restore_best_weights = True
)

In [ ]:
df = gl.coregister_data(data_path,'4')
df = df.drop(df[df['RGIId'].duplicated(keep = False)].index)
df = df.sample(frac = 1,random_state = 0)
df = df.reset_index().drop('index', axis = 1)
dft = df.copy()

trfeat = {}
trlabs = {}
tefeat = {}
telabs = {}

for n in df.index:
    mask = df.index.isin([n])
    trfeat[n] = df.loc[~mask].drop(['RGIId','Thickness'], axis = 1)
    tefeat[n] = df.loc[mask].drop(['RGIId','Thickness'], axis = 1)
    trlabs[n] = df['Thickness'].loc[~mask]
    telabs[n] = df['Thickness'].loc[mask]

model = {}
model_history = {}
normalizer = {}

for n in tqdm(df.index):
    model_path = os.path.join(LOO_path, str(n))
    if os.path.isdir(model_path) == True:
#         continue
        model[n] = gl.load_dnn_model(model_path)
    elif os.path.isdir(model_path) == False:
        set_global_determinism(seed=SEED)
        model[n] = run_model(model_path, n,l1 = 6,l2 = 2,loss = 'mae')
rgi_est_pth = os.path.join(LOO_path, 'rgi_est_raw.pkl')

RGI = gl.load_RGI(RGI_path)
rfp = RGI[list(df)[:-1]]

for n in tqdm(model.keys()):
    preds = pd.Series(
        model[n].predict(rfp.drop('RGIId',axis = 1)).flatten(), name = n
    )
    RGI = pd.concat([RGI,preds], axis = 1)
RGI.to_pickle(rgi_est_pth)

